In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches
import scipy
import pickle
import warnings

import sys
sys.path.append('..')
from construct_graph.graph import Graph, Flat
import construct_graph
import solve_laplace_operator

import sparseqr

def calculate_pde_eigenvalues(m, n):
    return np.sqrt(scipy.special.jn_zeros(m, n + 1)[-1]**2 / 2)

def generate_guesses(x, num_Vs):
    guesses = []
    for i in x:  
        if np.abs(i) < 1e-15:
            guesses.append(np.array([1e-15]))
        else:
            if num_Vs < 2000:
                guesses.append(np.linspace(0.8 * i, i + 0.2, 80))
            else:
                guesses.append(np.linspace(0.93 * i, i + 0.02, 60))
                
    return np.concatenate(guesses)

In [28]:
seed = 2
np.random.seed(seed)
g = construct_graph.random_delaunay_disc.Random_Delaunay_Disc(1000, fix_boundary=True, auto_plot=False)
eigenvalue_data = {}
eigenvalue_data[g.num_Vs] = {}

In [31]:
printerval = 1

guesses = np.concatenate((np.array([1e-15]), np.linspace(1.64, 1.65, 1)))[1:]
eigs = solve_laplace_operator.solve_laplace_operator.Eigenvalue_Calculator(g)
ks = eigs(guesses, printerval=printerval, max_steps=1000, tol=1e-12, solve_type="Newton trace estimation", estimation_type="Rademacher", num_vectors=20)
_, index = np.unique(np.round(ks, 6), return_index=True)
eigenvalues = ks[index]


Calculating eig number 0

Count = 1
k = 1.64

Count = 2
k = 1.6417618026076946

Count = 3
k = 1.6435339087942056

Count = 4
k = 1.645330431853842

Count = 5
k = 1.6470622428684745

Count = 6
k = 1.6488627992194262

Count = 7
k = 1.650634539098659

Count = 8
k = 1.6523623423686675

Count = 9
k = 1.6540171727331945

Count = 10
k = 1.655653423963602

Count = 11
k = 1.6572832914420514

Count = 12
k = 1.6587962476567761

Count = 13
k = 1.6604470173655521

Count = 14
k = 1.6620304346148806

Count = 15
k = 1.6636888192065626

Count = 16
k = 1.6651667369485317

Count = 17
k = 1.6664614270736031

Count = 18
k = 1.667508066934553

Count = 19
k = 1.6686748241065135

Count = 20
k = 1.6697574100897137

Count = 21
k = 1.6708004560600918

Count = 22
k = 1.6709065490206743

Count = 23
k = 1.6709101382825757

Count = 24
k = 1.6709103814846094

Count = 25
k = 1.670910515658921

Count = 26
k = 1.6709104742336638

Count = 27
k = 1.6709104653189484

Count = 28
k = 1.6709104670525248

Count = 29
k = 1.6709

In [25]:
eigenvalue_data[g.num_Vs][seed] = eigenvalues.copy()
print(np.round(eigenvalues, 4))

[1.6394]


In [26]:
eigenvalue_data[g.num_Vs][seed] / calculate_pde_eigenvalues(0, 0)

array([0.9641022])